# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [1]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [2]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [3]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [4]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [5]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [6]:
# Check the result:

A @ x - b

array([ 0.00000000e+00,  2.22044605e-16,  0.00000000e+00, -1.11022302e-16,
        0.00000000e+00,  0.00000000e+00, -2.08166817e-17,  0.00000000e+00,
        0.00000000e+00,  2.22044605e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [11]:
def yak(A, b, n_iter = 500):#запишем метод Якоби в функцию
    diag_1d = np.diag(A)
    B = -A.copy()
    np.fill_diagonal(B, 0)
    D = np.diag(diag_1d)
    invD = np.diag(1./diag_1d)
    BB = invD @ B 
    c = invD @ b
    x0 = np.ones(n)
    x = x0
    for _ in range(n_iter):
        x = BB @ x + c
    return A @ x - b, np.linalg.norm(BB)

In [12]:
yak(A, b, n_iter = 500)

(array([9.55484927e+233, 7.09031975e+233, 7.67741418e+233, 7.32356150e+233,
        1.17466462e+234, 1.26594291e+234, 1.04980466e+234, 8.23554964e+233,
        8.63864411e+233, 1.05440291e+234]),
 3.6414476395758366)

In [13]:
A = rndm.uniform(size=(n, n)) + np.diagflat([1]*n)#уберем доминирующий минор

In [14]:
yak(A, b, n_iter = 500)#норма больше едицы и все расходится

(array([3.95130724e+235, 3.10857306e+235, 3.45378888e+235, 1.67121199e+235,
        3.69823963e+235, 4.55336632e+235, 3.27907397e+235, 3.03138781e+235,
        3.74646963e+235, 3.63873934e+235]),
 3.761532616658444)

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [19]:
def seidel(A, b, n_iter = 500):
    x = np.zeros(len(A))
    x_new = np.copy(x)
    for i in range(len(A)):
        for j in range(i):
            s1 = sum(A[i][j] * x_new[j])#считаем сумму левых элементов
        for j in range(i + 1, n):
            s2 = sum(A[i][j] * x[j])#считаем сумму правых элементов
        x_new[i] = (b[i] - s1 - s2) / A[i][i]#выражаем неизвестные диагональные элементы
        x = x_new#
    return x

In [21]:
seidel(A, b, n_iter = 500)

array([ 0.70798109,  0.43830045,  0.26199732,  0.31326355,  0.16406624,
       -0.60747444, -0.06392456,  0.32575551, -0.64646053,  0.39024014])

# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [ ]:
# ... ENTER YOUR CODE HERE ...